In [2]:
!pip install praw
import praw
import pandas as pd
import datetime as dt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from bs4 import BeautifulSoup

You should consider upgrading via the 'c:\users\aditya kumar\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


[nltk_data] Downloading package stopwords to C:\Users\aditya
[nltk_data]     kumar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


# Setting up Reddit AuthO

In [3]:
reddit = praw.Reddit(client_id='WK7Bb4kSnRez8g',
                     client_secret='NYWBOS04ouTSPO0cKhsMLqs-SWQ',
                     user_agent='Reddit_India_Data')
india_subreddit = reddit.subreddit('india')

In [4]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]
post_dict = []

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def readableTime(created):
    return dt.datetime.utcfromtimestamp(created)


#To convert data elements into string objects
def toStr(item):
    return str(item)

_replace = re.compile('[/(){}\|@,;]')
badSymbols = re.compile('[^0-9a-z #+_]')
stopWords = set(stopwords.words('english'))

#To clean text within dataset

# sentences = 
lemmatizer = WordNetLemmatizer()
def cleanText(string):
    string = BeautifulSoup(string, "lxml").string
    string = string.lower() 
    string = _replace.sub(' ', string)
    string = badSymbols.sub('', string)
    string=nltk.sent_tokenize(string)
    words = [lemmatizer.lemmatize(word) for word in string if word not in set(stopwords.words('english'))]
    string= ' '.join(words)  
#     string = ' '.join(word for word in string.split() if word not in stopWords)
    return string

In [11]:
for flair in flairs:
    submissions = india_subreddit.search(f"flair_name:{flair}",limit=100)
    for submission in submissions:
        post_dict.append([submission.title,
                submission.id,
                submission.link_flair_text,
                submission.author,
                submission.url,
                submission.num_comments,
                submission.score,
                submission.created])
post_dict = pd.DataFrame(post_dict,columns=['title', 'id', 'flair', 'author', 'url', 'num_comments', 'score', 'created'])
topics = pd.DataFrame(post_dict)
#Sorting data based in ascending order of flair
topics.sort_values('flair', inplace=True)
#Changing Unix time to UTC time.
topics['timestamp'] = topics['created'].apply(readableTime)
del topics['created']
#Deleting any duplicate rows
topics.drop_duplicates(subset=None, inplace=True)
topics.to_csv('Data.csv', index=False)

# DB Cleaning and Text Pre Processing

In [12]:
df = pd.read_csv('Data.csv')
df.head()
df['title'] =df['title'].apply(toStr)
df['title'] =df['title'].apply(cleanText)
feature_Grp = df['title'] + df['url']
df = df.assign(feature_Grp = feature_Grp)
df.to_csv('Final_formatted_data.csv', index=False)

In [13]:
df = pd.read_csv('Final_formatted_data.csv')


In [14]:
df.head()

,title,id,flair,author,url,num_comments,score,timestamp,feature_Grp
0,i am a lesser known indian author my 2nd book ...,5unqxo,AMA,legalindia,https://www.reddit.com/r/india/comments/5unqxo...,145,93,2017-02-17 18:27:13,i am a lesser known indian author my 2nd book ...
1,i am sanjeev sabhlok joined ias in 1982 and r...,9mxy2y,AMA,sabhlok,https://www.reddit.com/r/india/comments/9mxy2y...,304,187,2018-10-10 09:30:47,i am sanjeev sabhlok joined ias in 1982 and r...
2,hey reddit india we are the founding editors ...,8idukp,AMA,thewire_in,https://www.reddit.com/r/india/comments/8idukp...,293,418,2018-05-10 10:26:40,hey reddit india we are the founding editors ...
3,hi im shashi tharoor ask me anything on india ...,626l9b,AMA,shashitharoor2017,https://www.reddit.com/r/india/comments/626l9b...,496,1602,2017-03-29 13:25:30,hi im shashi tharoor ask me anything on india ...
4,ama on aadhaar with kiran jonnalagadda anivar...,7sw2bj,AMA,kumbhakaran,https://www.reddit.com/r/india/comments/7sw2bj...,458,313,2018-01-25 13:31:13,ama on aadhaar with kiran jonnalagadda anivar...
